In [5]:
import math
import scipy
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
from tqdm import tqdm   # gives progression bar
import pickle
%matplotlib inline


In [6]:
# constants in SI
m = 9.10938291E-31 # mass of electron
q = 1.60217657E-19 # charge of electron
ε = 8.85418782E-12 # permittivity of free space
h = 6.62606957E-34 # plancks magical number
hbar = 1.054571800E-34 # reduced planck
Rydberg = 2.1787E-18 # Rydberg energy (in joules)
avogadro = 6.02214179E23
# units
Å = 1E-10 # m to angstrom
k = 1.38064852E-23

In [7]:
def diffusivity(thermal_conductivity, d, molecular_weight,heat_cap):
    # diffusivity = thermal_conductivity / (density*heat capacity)
    density = (molecular_weight/avogadro) / (d**3) # g/m^3 
    heat_cap = heat_capacity / molecular_weight # J/g/K
    return thermal_conductivity / (density*heat_cap) # m^2/s

## Finite difference method - anisotropic

- We are going to use a finite difference method (explicit forward euler scheme) to solve numerically
- Following https://thebrickinthesky.wordpress.com/2013/08/17/maths-with-python-3-diffusion-equation/
- Using theory: https://hinderedsettling.com/2015/02/06/exploring-the-diffusion-equation-with-python/
- Generalisation to higher dimensionalities: http://web.mit.edu/1.061/www/dream/THREE/THREETHEORY.PDF

In [8]:
def assess_stability(D_max,dx,dt):
    # Key dimensionless parameter which determines stability is
    F = (D_max*dt)/(dx*dx)
    print ("F is {}".format(F))
# F needs to be less than 0.5 for a stable solution

In [14]:
def initial_conditions(polaron_radius, temperature, dx, dy, dz, dt, nx, ny, nz, nt):   
    # Create p matrix which will contain the numerical solution.
    p=np.zeros([nx,ny,nz,nt])

    initial_hot_count = 0
    for y in range(0,ny-1):
        for x in range(0,nx-1):
            for z in range(0,nz-1):
                if (( ((dx*(x - round(nx/2)))**2) + ((dy*(y - round(ny/2)))**2)+((dz*(z - round(nz/2)))**2))**(0.5)) < polaron_radius :
                    initial_hot_count += 1
                    p[x,y,z,0] = temperature # we just need temperature above RT: difference is what matters
                else: 
                    p[x,y,z,0] = 0
    print("initial hot cells:{}".format(initial_hot_count))
    return p

In [15]:
# use "fridge conditions": the edges are set to zero.
def step_through(p, Dx, Dy, Dz, dx, dy, dz, dt, nx, ny, nz, nt):
    for m in tqdm(range(1,nt)):
    # Take time slices and apply central difference formula
        p[1:nx-1,1:ny-1,1:nz-1,m]=p[1:nx-1,1:ny-1,1:nz-1,m-1]+(dt*Dx*(p[2:nx,1:ny-1,1:nz-1,m-1]-2*p[1:nx-1,1:ny-1,1:nz-1,m-1]+p[0:nx-2,1:ny-1,1:nz-1,m-1])/np.power(dx,2))+(Dy*dt*(p[1:nx-1,2:ny,1:nz-1, m-1]-2*p[1:nx-1,1:ny-1,1:nz-1, m-1]+p[1:nx-1,0:ny-2,1:nz-1, m-1])/np.power(dy,2)) + (Dz*dt*(p[1:nx-1,1:ny-1,2:nz,m-1]-2*p[1:nx-1,1:ny-1,1:nz-1,m-1]+p[1:nx-1,1:ny-1,0:nz-2,m-1])/np.power(dz,2))
    
    return p

In [11]:
def initial_conditions_multiple_hotspots(polaron_radius, temperature, dx, dy, dz, dt, nx, ny, nz, nt, step, number):   
    # Create p matrix which will contain the numerical solution.
    p=np.zeros([nx,ny,nz,nt])

    initial_hot_count = 0
    steps = [x*step for x in range(1,number+1)]
    for a in steps:
        x_position = a
        for b in steps:
            y_position = b
            for c in steps:
                z_position = c
                for y in range(0,ny-1):
                    for x in range(0,nx-1):
                        for z in range(0,nz-1):
                            if (( ((dx*(x - round(nx*x_position)))**2) + ((dy*(y - round(ny*y_position)))**2)+((dz*(z - round(nz*z_position)))**2))**(0.5)) < polaron_radius :
                                initial_hot_count += 1
                                p[x,y,z,0] = temperature # we just need temperature above RT: difference is what matters

    print("initial hot cells:{}".format(initial_hot_count))
    return p

In [12]:
# use periodic boundary conditions
def step_through_PBCs(p, Dx, Dy, Dz,  dx, dy, dz, dt, nx, ny, nz, nt):
    for m in tqdm(range(1,nt)):
    # Take time slices and apply central difference formula - using np.roll and np.rollaxis now for cycling forwards and backwards respectively through the array to achieve PBC's.
        p[:,:,:,m]=p[:,:,:,m-1]+(dt*Dx*(np.roll(p[:,:,:,m-1],nx-1,axis=0)-2*p[:,:,:,m-1]+np.roll(p[:,:,:,m-1],1,axis=0))/np.power(dx,2)) + (dt*Dy*(np.roll(p[:,:,:, m-1],ny-1,axis=1)-2*p[:,:,:, m-1]+np.roll(p[:,:,:, m-1],1,axis=1))/np.power(dy,2)) + (dt*Dx*(np.roll(p[:,:,:,m-1],nz-1,axis=2)-2*p[:,:,:,m-1]+np.roll(p[:,:,:,m-1],1,axis=2))/np.power(dz,2))
    return p

In [13]:
def calculate_temperature_polaron(nx, ny, nz, dx, dy, dz, dt, p, polaron_radius, time):
    temp = 0
    count = 0
    for y in range(0,ny-1):
        for x in range(0,nx-1):
            for z in range(0,nz-1):
                if (( ((dx*(x - round(nx/2)))**2) + ((dy*(y - round(ny/2)))**2)+((dz*(z - round(nz/2)))**2))**(0.5)) < polaron_radius :
                    temp += p[x,y,z,time]
                    count += 1
    return temp/count

## Gaussian convolution - isotropic

Note that the `crank_diffusion_sphere` was the method used in the hot carrier cooling paper (2017)

- Theory: http://hplgit.github.io/num-methods-for-PDEs/doc/pub/diffu/sphinx/._main_diffu001.html
- Theory: http://web.mit.edu/course/1/1.061/www/dream/THREE/THREETHEORY.PDF
- https://www.amherst.edu/media/view/103447/original/Diffusion%2Bpart%2B1%2BPDF.pdf
- http://www1.maths.leeds.ac.uk/~kersale/Teach/M3414/Notes/m3414_1.pdf
- https://math.stackexchange.com/questions/833002/distance-between-two-points-in-spherical-coordinates
- https://www.khanacademy.org/math/multivariable-calculus/integrating-multivariable-functions/triple-integrals-a/a/triple-integrals-in-spherical-coordinates

In [151]:
def gaussian_solution_cube(initial_temperature, polaron_radius, diffusivity, x1, y1, z1, time ):
    temperature = integrate.tplquad(diffusion_equation_cube,-polaron_radius,polaron_radius, lambda x: -polaron_radius,lambda x: polaron_radius,lambda x,y: -polaron_radius,lambda x,y: polaron_radius,args=(initial_temperature,diffusivity,x1,y1,z1,time))   
    return temperature

def diffusion_equation_cube(x,y,z,initial_temperature,diffusivity, x1, y1, z1, time):
    return initial_temperature*(1/(np.sqrt((4*math.pi*diffusivity*time)**3)))*math.exp(-((((x1-x)**2)+(y1-y)**2)+(z1-z)**2)/(4*diffusivity*time))

## WIP
def gaussian_solution_sphere(initial_temperature, polaron_radius, diffusivity, x, time ):
    temperature = integrate.quad(diffusion_equation_sphere,0,polaron_radius, args=(initial_temperature,diffusivity, x, time))   
    return temperature

## WIP
def diffusion_equation_sphere(r,initial_temperature,diffusivity, x, time):
    return (initial_temperature*(1/(np.sqrt(((diffusivity*time)**3)*4*math.pi)))*r*r*math.exp(-(((x-r)**2))/(4*diffusivity*time)))

def crank_diffusion_sphere(initial_temperature,polaron_radius,D,position,time):
    # from Crank's The Mathematics of Diffusion
    return 0.5*initial_temperature*(math.erf((polaron_radius+position)/np.sqrt(4*D*time)) + math.erf((polaron_radius-position)/np.sqrt(4*D*time))) - ((initial_temperature/position)*np.sqrt(D*time/math.pi)*(math.exp(-((polaron_radius-position)**2)/(4*D*time))-math.exp(-((polaron_radius+position)**2)/(4*D*time))))

## Point heat source

In [107]:
def point_source(heat,diffusivity,time,position,radius):
    # heat is diffusing not temperature, it returns a concentration of energy

    return (heat)*(1/((4*math.pi*diffusivity*time)**(3/2)))*math.exp(-((position**2)/(4*diffusivity*time)))
